In [ ]:
from forward import solve_forward
from simulation import Simulation
import numpy as np
import mat73
from net import Region_CNN

In [ ]:
sources = np.load('/media/thanos/Elements/thanos/sim_data/sim_type_1/region_dataset/Region 2/region_2_sources.npy')
eeg = np.load('/media/thanos/Elements/thanos/sim_data/sim_type_1/region_dataset/Region 2/region_2_eeg.npy')
fwd = solve_forward()
eeg_topos = mat73.loadmat('/media/thanos/Elements/thanos/sim_data/sim_type_1/region_dataset/Region 2/region_2_eeg_topos.mat')['eeg_topos']
eeg_topos = eeg_topos.transpose(2, 0, 1)

sim = Simulation(fwd=fwd, source_data=sources, eeg_data=eeg)
region_cnn = Region_CNN(sim=sim, eeg_topographies=eeg_topos)

region_cnn.load_nn('/media/thanos/Elements/thanos/nn_trained/Regions Cnn/Region-2/region_2_cnn_1.h5')

In [ ]:
topos_eval = mat73.loadmat('/media/thanos/Elements/thanos/sim_data/sim_type_1/Regions-Evaluate-Sources/Region 2/region_2_topos_eval.mat')['eeg_topos']
topos_eval = topos_eval.transpose(2, 0, 1)

In [ ]:
predicted_sources = region_cnn.predict(eeg=topos_eval).T

In [ ]:
np.save('../../Downloads/pred_sources.npy',predicted_sources)